## Dataframe for MHW metrics

In [1]:
import dask
import pandas as pd
import netCDF4
import xarray as xr
import numpy as np
import dask.array as da
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import matplotlib.pyplot as plt
import matplotlib.cm as cm

print('loaded libraries')

loaded libraries


In [2]:
dir_path = "/pub/hfdrake/datasets/CM4_MHW_blobs/"
mt_path = "/pub/mariant3/WarmWaterMasses/data/"
ds = xr.open_dataset(f"{dir_path}/data/ocean_daily_cmip.01860101-01901231.tos.nc", chunks={'time':100})
ds_region = ds.sel(xh=slice(-130, -70), yh=slice(8, 38)) #.sel(xh = slice(-138, 0), yh = slice(8, 49))
static = xr.open_dataset(f"{dir_path}/data/ocean_daily_cmip.static.nc")
static_region = static.sel(xh=slice(-130, -70), yh=slice(8, 38))
#ds_static_region = xr.merge([static_region,ds_region])
#labels = xr.open_dataset("/pub/mariant3/WarmWaterMasses/data/01860315-01891101-ocetrac-v3-labels-region.nc")
ocetrac_labels = xr.open_mfdataset(f"{mt_path}/ocetracv5/ocetrac-v5-processed/ocetrac-v5-blobs-tos-t2*.nc", chunks={'time':1})

In [3]:
labels = ocetrac_labels['blobs'].load()

In [4]:
ds_labels_region = xr.merge([ds_region,labels],join='inner')
ds_labels_static_region = xr.merge([static_region,ds_labels_region],join='inner') #previously ds_static_region

In [5]:
ids = np.unique(labels)
ids = np.array([id for id in ids if ~np.isnan(id)])
ids

array([  37.,   55.,   66.,   76.,   89.,   95.,   98.,  101.,  104.,
        107.,  108.,  124.,  125.,  128.,  131.,  132.,  133.,  138.,
        141.,  142.,  145.,  148.,  152.,  153.,  167.,  179.,  181.,
        182.,  183.,  187.,  188.,  194.,  198.,  199.,  202.,  211.,
        214.,  217.,  224.,  225.,  231.,  232.,  233.,  234.,  236.,
        238.,  240.,  244.,  254.,  255.,  256.,  268.,  271.,  273.,
        279.,  281.,  286.,  287.,  289.,  290.,  291.,  293.,  296.,
        301.,  306.,  319.,  321.,  324.,  329.,  330.,  332.,  335.,
        337.,  340.,  346.,  347.,  349.,  350.,  351.,  354.,  357.,
        358.,  359.,  360.,  374.,  375.,  376.,  382.,  388.,  458.,
        477.,  483.,  495.,  497.,  501.,  504.,  511.,  513.,  519.,
        521.,  522.,  526.,  529.,  531.,  532.,  538.,  540.,  542.,
        546.,  548.,  549.,  553.,  554.,  556.,  558.,  562.,  564.,
        569.,  574.,  577.,  582.,  583.,  584.,  585.,  586.,  587.,
        588.,  589.,

In [6]:
num_events = len(ids)
num_events

303

In [7]:
for id in ids:
    print(id)
    #print(i)
    event = labels.where(labels==id, drop=True)
    #print(event)
    event_mask = event == event

37.0
55.0
66.0
76.0
89.0
95.0
98.0
101.0
104.0
107.0
108.0
124.0
125.0
128.0
131.0
132.0
133.0
138.0
141.0
142.0
145.0
148.0
152.0
153.0
167.0
179.0
181.0
182.0
183.0
187.0
188.0
194.0
198.0
199.0
202.0
211.0
214.0
217.0
224.0
225.0
231.0
232.0
233.0
234.0
236.0
238.0
240.0
244.0
254.0
255.0
256.0
268.0
271.0
273.0
279.0
281.0
286.0
287.0
289.0
290.0
291.0
293.0
296.0
301.0
306.0
319.0
321.0
324.0
329.0
330.0
332.0
335.0
337.0
340.0
346.0
347.0
349.0
350.0
351.0
354.0
357.0
358.0
359.0
360.0
374.0
375.0
376.0
382.0
388.0
458.0
477.0
483.0
495.0
497.0
501.0
504.0
511.0
513.0
519.0
521.0
522.0
526.0
529.0
531.0
532.0
538.0
540.0
542.0
546.0
548.0
549.0
553.0
554.0
556.0
558.0
562.0
564.0
569.0
574.0
577.0
582.0
583.0
584.0
585.0
586.0
587.0
588.0
589.0
597.0
598.0
603.0
610.0
615.0
618.0
621.0
625.0
628.0
629.0
632.0
633.0
638.0
640.0
641.0
642.0
643.0
645.0
646.0
651.0
653.0
661.0
666.0
670.0
680.0
683.0
688.0
689.0
693.0
700.0
705.0
709.0
711.0
713.0
714.0
715.0
722.0
723.0
725.0
728.0

In [9]:
#Define ssta
ssta = ds_labels_static_region.tos

In [10]:
import os, glob
import xarray as xr
import numpy as np
import pandas as pd
import warnings
from skimage.measure import label as label_np
from skimage.measure import regionprops 

try:
    from tqdm.auto import tqdm
    tqdm_avail = True
except:
    warnings.warn(
        "Optional dependency `tqdm` not found. This will make progressbars a lot nicer. \
    Install with `conda install -c conda-forge tqdm`"
    )
    tqdm_avail = False
    
def _wrap(labels):
    ''' Impose periodic boundary and wrap labels, then reorder the labels'''
    first_column = labels[..., 0]
    last_column = labels[..., -1]

    stacked = first_column.stack(z=['time','yh'])
    unique_first = np.unique(stacked[stacked.notnull()])

    # This loop iterates over the unique values in the first column, finds the location of those values in 
    # the first columnm and then uses that index to replace the values in the last column with the first column value
    for i in enumerate(unique_first):
        first = np.where(first_column == i[1])
        last = last_column[first[0], first[1]]
        stacked = last.stack(z=['time','yh'])
        bad_labels = np.unique(stacked[stacked.notnull()])
        replace = np.isin(labels, bad_labels)
        labels = labels.where(replace==False, other=i[1])
    
    labels = labels.fillna(0)
    labels_wrapped = np.unique(labels, return_inverse=True)[1].reshape(labels.shape)
    labels_wrapped = xr.DataArray(labels_wrapped, dims=labels.dims, coords=labels.coords)

    return labels_wrapped

def _get_labels(binary_images):
    '''function used to label binary images at each time step using skimage.measure.label'''
    blobs_labels = label_np(binary_images, background=0)
    return blobs_labels
    
def _get_centroids(sub_labels):
    '''This function uses skimage.measure.regionprops to find the centroids of objects assigned 
    to each unique label'''
    props = regionprops(sub_labels.astype('int'))
    centroids = [(float(sub_labels.yh[round(p.centroid[0])].values),
                  float(sub_labels.xh[round(p.centroid[1])].values)) for p in props]
    for i in range(0,len(centroids)):
        if centroids[i][1] >= 359.875:
            centroids[i] = (centroids[i][0], list(centroids[i])[1] - 359.875)
    
    return centroids

def _get_intensity_area(event, ssta, mhw):
    '''Calculates event intensities and area at each time interval using anomaly data and coordinates 
    cooresponding to the event.'''
    
    event_ssta = ssta.where(event>0, drop=True)        
    event_ssta_mask = event_ssta == event_ssta
    #mhw['intensity_mean'].append(event_ssta.mean(('yh','xh')).values)
    #mhw['intensity_max'].append(event_ssta.max(('yh','xh')).values) 
    #mhw['intensity_min'].append(event_ssta.min(('yh','xh')).values)
    mhw['intensity_mean'].append((ds_labels_static_region.tos).where(event_ssta_mask).mean(['xh','yh']).values)
    mhw['intensity_min'].append((ds_labels_static_region.tos).where(event_ssta_mask).min(['xh','yh']).values)
    mhw['intensity_max'].append((ds_labels_static_region.tos).where(event_ssta_mask).max(['xh','yh']).values)
   
    #event_temperature_max = (ds_static_region.tos).where(event_mask).sel(xh = slice(-138, 0), yh = slice(8, 49)).max(['xh','yh'])
 
    mhw['intensity_cumulative'].append(np.nansum(event_ssta))
    coords = event.stack(z=('yh','xh'))
    coord_pairs = [(coords.isel(time=t[0]).dropna(dim='z', how='any').z.yh.values, 
                      coords.isel(time=t[0]).dropna(dim='z', how='any').z.xh.values) for t in enumerate(event.time)]

    mhw['coords'].append(coord_pairs)
    
    # Calculate weighted cell area assuming 0.25º resolution data and 111 km per degree of latitude
    
    #for id in ids:
        #print(id)
        #print(i)
        #events = labels.where(labels==id, drop=True)
        #event_mask = events == id
   # event_area
    #event_mask = events == event
    event_ssta_mask = event_ssta == event_ssta
    #print(event_ssta == event_ssta)
    #print(event_ssta.areacello.sum(('yh','xh')).values)
    event_area = (ds_labels_static_region.areacello).where(event_ssta_mask).sum(['xh','yh'])
    # event_area = (ds_static_region.areacello).where(event_ssta_mask).sel(xh = slice(-138, 0), yh = slice(8, 49)).sum(['xh','yh'])
    #print(event_area.values)
    cell_area = event_area.values
    mhw['area'].append(cell_area)


    #y, x = zip(*coord_pairs)
    #dlon = [np.cos(y[c]*np.pi/180)*(111*.25) for c in np.arange(0, len(coord_pairs))]
    #dlat = (111*.25) * np.ones(len(dlon))
    #cell_area = [np.sum(dlon[c]*dlat[c]) for c in np.arange(0, len(coord_pairs))]
    #mhw['area'].append(cell_area)

    
    return mhw

In [11]:
def _get_metrics(event, ssta):
    '''
    Creates a Pandas DataFrame of event attributes.
    
    Parameters
    ----------
      event : xarray.DataArray   
              Image set containing only objects corresponding to the event of interest. 
              Dimensions should be ('time', 'lat', 'lon')
              
      ssta  : xarray.DataArray
              Temperature vector [1D numpy array of length T]
    
    Returns
    -------
    
    mhw : pandas.DataFrame
          Marine heat wave event attributes. The keys listed below are 
          are contained within the dataset.
 
        'id'                     Unique label given to the MHW [int]
        'date'                   Dates corresponding to the event [datetime format]
        'coords'                 Latitude and longitude of all points contained in the event [tuple(lat,lon)]
        'centroid'               Center of each object contained in the event [tuple(lat,lon)]
        'duration'               Duration of event [months]
        'intensity_max'          Maximum intensity at each time interval [degC]
        'intensity_mean'         Mean intensity at each time interval [degC]
        'intensity_min'          Minimum intensity at each time interval [degC]
        'intensity_cumulative'   Cumulated intensity over the entire event [degC months]
        'area'                   Area of the event at each time interval [km2]
        
    '''
    
    # Initialize dictionary 
    mhw = {}
    mhw['id'] = [] # event label
    mhw['date'] = [] # datetime format
    mhw['coords'] = [] # (lat, lon)
    mhw['centroid'] = []  # (lat, lon)
    mhw['duration'] = [] # [months]
    mhw['intensity_max'] = [] # [deg C]
    mhw['intensity_mean'] = [] # [deg C]
    mhw['intensity_min'] = [] # [deg C]
    mhw['intensity_cumulative'] = [] # [deg C]
    mhw['area'] = [] # [km2]

    # TO ADD:
    # mhw['rate_onset'] = [] # [deg C / month]
    # mhw['rate_decline'] = [] # [deg C / month]

    mhw['id'].append(int(np.nanmedian(event.values)))
    mhw['date'].append(event.time.values.astype('datetime64[D]'))
    mhw['duration'].append(event.time.shape[0])

    # Turn images into binary
    binary_event = event.where(event>=0, other=0)
    binary_event = binary_event.where(binary_event==0, other=1)
      
    sub_labels = xr.apply_ufunc(_get_labels, binary_event,
                                input_core_dims=[['yh', 'xh']],
                                output_core_dims=[['yh', 'xh']],
                                output_dtypes=[binary_event.dtype],
                                vectorize=True,
                                dask='parallelized')
    
    # Turn background to NaNs
    sub_labels = xr.DataArray(sub_labels, dims=binary_event.dims, coords=binary_event.coords)
    sub_labels = sub_labels.where(sub_labels>0, drop=False, other=np.nan) 

    # The labels are repeated each time step, therefore we relabel them to be consecutive
    for p in range(1, sub_labels.shape[0]):
        sub_labels[p,:,:] = sub_labels[p,:,:].values + sub_labels[p-1,:,:].max().values
    
    sub_labels_wrapped = _wrap(sub_labels)
    
    mhw = _get_intensity_area(event, ssta, mhw)
    
 # '  centroid = []
 #   for s in np.arange(0, sub_labels_wrapped.shape[0]):
 #       lx = sub_labels_wrapped.isel(time=s)
 #       east = lx.where(lx.xh < 180, drop=True)
 #       east['xh'] = labels.xh
 #       append_east = xr.concat([lx.where(lx.xh >= 180, drop=True), east], dim="xh")
#        centroid.append(_get_centroids(append_east))
#    mhw['centroid'].append(centroid)'
    
    mhw = pd.DataFrame(dict([(name, pd.Series(data)) for name,data in mhw.items()]))
#     mhw.to_csv('df_'+str(mhw['id'].values[0]).zfill(4)+'.csv', index=False)
    return mhw


In [ ]:
dataframes = []  
for i in range(0,num_events):
    print(ids[i])
    event = labels.where(labels==ids[i], drop=True)
    df = _get_metrics(event, ssta)
    dataframes.append(df)

37.0
55.0
66.0
76.0
89.0
95.0
98.0
101.0
104.0
107.0
108.0
124.0
125.0
128.0
131.0
132.0
133.0
138.0
141.0
142.0
145.0
148.0
152.0
153.0
167.0
179.0
181.0
182.0
183.0
187.0
188.0
194.0
198.0
199.0
202.0
211.0
214.0
217.0
224.0
225.0
231.0
232.0
233.0
234.0
236.0
238.0
240.0
244.0
254.0
255.0
256.0
268.0
271.0
273.0
279.0
281.0
286.0
287.0
289.0
290.0
291.0
293.0
296.0
301.0
306.0
319.0
321.0
324.0
329.0
330.0
332.0
335.0
337.0
340.0
346.0
347.0
349.0
350.0
351.0
354.0
357.0
358.0
359.0


In [ ]:
dff = pd.concat(dataframes, ignore_index=True)
dff

In [28]:
import pickle

In [29]:
dff.loc[3]["date"]

array(['0186-07-26', '0186-07-27', '0186-07-28', '0186-07-29',
       '0186-07-30', '0186-07-31', '0186-08-01', '0186-08-02',
       '0186-08-03', '0186-08-04', '0186-08-05', '0186-08-06',
       '0186-08-07', '0186-08-08', '0186-08-09', '0186-08-10',
       '0186-08-11', '0186-08-12', '0186-08-13', '0186-08-14',
       '0186-08-15', '0186-08-16', '0186-08-17', '0186-08-18',
       '0186-08-19', '0186-08-20', '0186-08-21', '0186-08-22',
       '0186-08-23', '0186-08-24', '0186-08-25'], dtype='datetime64[D]')

In [35]:
print(f"{mt_path}ocetracv5/ocetrac-v5-processed")

/pub/mariant3/WarmWaterMasses/data/ocetracv5/ocetrac-v5-processed


In [36]:
dff.to_pickle(f"{mt_path}ocetracv5/ocetrac-v5-processed/ocetrac-v5-blobs-tos-t1-df-r3-msq75-01860315-01891101-manso.pkl")

In [39]:
test_df = pd.read_pickle(f"{mt_path}ocetracv5/ocetrac-v5-processed/ocetrac-v5-blobs-tos-t1-df-r3-msq75-01860315-01891101-manso.pkl")
test_df

,id,date,coords,centroid,duration,intensity_max,intensity_mean,intensity_min,intensity_cumulative,area
0,15,"[0186-05-09, 0186-05-10, 0186-05-11, 0186-05-1...","[([10.687478569214495, 10.687478569214495, 10....",NaN,16,"[30.577562, 30.524511, 30.282787, 29.90784, 29...","[29.438, 29.407696, 29.413874, 29.361107, 29.3...","[28.607817, 28.37463, 28.442465, 28.612474, 28...",5.406714e+05,"[380071300000.0, 696309600000.0, 792937000000...."
1,18,"[0186-05-30, 0186-05-31, 0186-06-01, 0186-06-0...","[([10.933041787532273, 10.933041787532273, 10....",NaN,13,"[30.057089, 30.454035, 30.46838, 30.451525, 30...","[29.311945, 29.442247, 29.538956, 29.57242, 29...","[28.448698, 28.309492, 28.496428, 28.540184, 2...",3.501574e+05,"[340643500000.0, 483869520000.0, 622780100000...."
2,34,[0186-07-20],"[([18.66104935115552, 18.66104935115552, 18.66...",NaN,1,[31.270472],[29.813925],[27.587831],1.302869e+04,[285375530000.0]
3,37,"[0186-07-26, 0186-07-27, 0186-07-28, 0186-07-2...","[([25.82802282930681, 25.82802282930681, 25.82...",NaN,31,"[31.408276, 31.437366, 31.503633, 31.628426, 3...","[29.645166, 29.672773, 29.730291, 29.838388, 2...","[28.889091, 28.83733, 28.804953, 28.842989, 28...",1.214388e+06,"[287833980000.0, 352467260000.0, 351324600000...."
4,38,"[0186-08-03, 0186-08-04, 0186-08-05, 0186-08-0...","[([20.54502229317784, 20.54502229317784, 20.54...",NaN,30,"[31.636108, 31.620401, 31.84838, 32.26465, 32....","[29.981346, 29.97524, 30.070566, 30.144043, 30...","[27.698223, 27.66954, 27.685774, 27.989813, 27...",8.841995e+05,"[298516220000.0, 322080870000.0, 325828940000...."
...,...,...,...,...,...,...,...,...,...,...
58,215,[0189-08-16],"[([8.22163897284192, 8.22163897284192, 8.22163...",NaN,1,[29.97663],[29.367453],[26.959581],8.810236e+02,[22742043000.0]
59,220,"[0189-09-03, 0189-09-04, 0189-09-05]","[([21.012499247321283, 21.012499247321283, 21....",NaN,3,"[32.339622, 32.705437, 33.077084]","[30.461796, 30.413374, 30.594948]","[28.013126, 28.05437, 27.899927]",4.378177e+04,"[271604150000.0, 334425800000.0, 304767660000.0]"
60,222,"[0189-09-13, 0189-09-14, 0189-09-15, 0189-09-1...","[([8.468991599524765, 8.468991599524765, 8.716...",NaN,12,"[31.195784, 31.084225, 30.90994, 30.76646, 30....","[29.502768, 29.43599, 29.426449, 29.510523, 29...","[28.625645, 28.597101, 28.455843, 28.528925, 2...",2.661274e+05,"[680705900000.0, 892234100000.0, 1067988700000..."
61,224,"[0189-09-28, 0189-09-29, 0189-09-30]","[([16.03642806679678, 16.03642806679678, 16.03...",NaN,3,"[30.51143, 30.597382, 30.699276]","[29.210794, 29.263695, 29.27209]","[28.467686, 28.464714, 28.709913]",4.173915e+04,"[309382120000.0, 366004760000.0, 302475970000.0]"
